# Benchmarks

In [58]:
using Revise
using BenchmarkTools
using DataFrames
using Plots
using Random
using Unitful

In [59]:
include("../src/MyLL.jl")
include("../src/MyV.jl")
using .MyLL
using .MyV

## Singly
### Comparing Singly Linked Lists to Vectors

1. Vary the size of a linked list and
   1. append it to a fixed size linked list
   2. prepend it to a fixed size linked list
2. Vary the size of a vector and
   1. append it to a fixed size vector i.e. copy both to a new vector
   2. prepend it to a fixed size vector i.e. copy both to a new vector
3. Stack operations, lists vs vectors (how?)

## Doubly
### Comparing Singly vs Doubly Linked Lists

* Keep a list of n elements and perform k remove and add operations
* Remove and add it back i.e. 
  * item = popat!(list, i) and then
  * pushfirst(list, item)!

create a vector with length k
v[i] = random number from 1:n

choose a random sequence of indexes from the vector

In [80]:
function doublybench(n, k)

    sequence_vector = createrandomvector(k, n)
    singlylinked_list = createrandom_sllist(n)
    doublylinked_list = createrandom_dllist(n)

    sll_time = @elapsed begin
        for i in eachindex(sequence_vector)
            pushfirst!(singlylinked_list, popat!(singlylinked_list, sequence_vector[i]))
        end
    end

    dll_time = @elapsed begin
        for i in eachindex(sequence_vector)
            pushfirst!(doublylinked_list, popat!(doublylinked_list, sequence_vector[i]))
        end
    end
    return n, k, sll_time*u"ns", dll_time*u"ns" # u stands for unit and ns for nanosecunds
end

doublybench (generic function with 1 method)

In [90]:
k = 1000 # k operations

benchvector = Vector(undef, 0)
for n = 100000:100000:1000000 
    push!(benchvector, doublybench(n, k))
end

In [91]:
for timedoperation in benchvector
    println(timedoperation)
end

(100000, 1000, 0.180947266 ns, 0.090932313 ns)
(200000, 1000, 0.363947781 ns, 0.19382245 ns)
(300000, 1000, 0.880057274 ns, 0.343958595 ns)
(400000, 1000, 0.91118367 ns, 0.477164611 ns)
(500000, 1000, 1.358067947 ns, 0.773830694 ns)
(600000, 1000, 2.398195865 ns, 0.935602868 ns)
(700000, 1000, 2.442772615 ns, 1.50625226 ns)
(800000, 1000, 2.546316137 ns, 1.266844354 ns)
(900000, 1000, 3.618910839 ns, 1.415197444 ns)
(1000000, 1000, 3.186418639 ns, 1.998141858 ns)



## TODO: Data manipulation
* Ratio,  
  * Growth (within list)
  * Difference between lists
* Plots
  * do this with Plots.jl
* n and k in a more readable way
  * e.g. 10 000 instead of 10000 
* Manipulate time units. 
  * E.g. from ns to ms. Do this with Unitful


In [92]:
df = DataFrame([[benchvector[k][kk] for k in 1:length(benchvector)] for kk in 1:length(benchvector[1])], [:n_elements, :k_operations, :SinglyLinkedList, :DoublyLinkedList])

,n_elements,k_operations,SinglyLinkedList,DoublyLinkedList
,Int64,Int64,Quantity…,Quantity…
1,100000,1000,0.180947 ns,0.0909323 ns
2,200000,1000,0.363948 ns,0.193822 ns
3,300000,1000,0.880057 ns,0.343959 ns
4,400000,1000,0.911184 ns,0.477165 ns
5,500000,1000,1.35807 ns,0.773831 ns
6,600000,1000,2.3982 ns,0.935603 ns
7,700000,1000,2.44277 ns,1.50625 ns
8,800000,1000,2.54632 ns,1.26684 ns
9,900000,1000,3.61891 ns,1.4152 ns


In [ ]:
fig = plot()

In [67]:
# testar att det funkar
k = 5
n = 10
sequence_vector = createrandomvector(k, n)
singlylinked_list = createrandom_sllist(n)
doublylinked_list = createrandom_dllist(n)

println(sequence_vector)
println("sll before")
println(singlylinked_list)
println("after before")
for i in eachindex(sequence_vector)
    pushfirst!(singlylinked_list, popat!(singlylinked_list, sequence_vector[i]))
    println(singlylinked_list)
end

println(sequence_vector)
println("dll before")
println(doublylinked_list)
println("dll before")
for i in eachindex(sequence_vector)
    pushfirst!(doublylinked_list, popat!(doublylinked_list, sequence_vector[i]))
    println(doublylinked_list)
end

[8, 6, 10, 7, 3]
sll before
86 22 58 41 27 80 11 82 68 62 
after before
82 86 22 58 41 27 80 11 68 62 
27 82 86 22 58 41 80 11 68 62 
62 27 82 86 22 58 41 80 11 68 
41 62 27 82 86 22 58 80 11 68 
27 41 62 82 86 22 58 80 11 68 
[8, 6, 10, 7, 3]
dll before
80 33 79 11 89 17 46 83 37 57 
dll before
83 80 33 79 11 89 17 46 37 57 
89 83 80 33 79 11 17 46 37 57 
57 89 83 80 33 79 11 17 46 37 
11 57 89 83 80 33 79 17 46 37 
89 11 57 83 80 33 79 17 46 37 
